In [ ]:
# Setup for Google Colab (optional)
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    # Install required packages
    !pip install -q py4DSTEM hyperspy scikit-image matplotlib numpy scipy
    
    # Set up file handling
    from google.colab import files
    print("Colab setup complete. Use files.upload() to upload data, files.download() to download results.")
else:
    print("Running in local environment")

<a href="https://colab.research.google.com/github/NU-MSE-LECTURES/465-WINTER2026/blob/main/Week_02/code_examples/example_03_calibration_practices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 02 Example 3: Calibration Best Practices and Troubleshooting

This notebook provides practical guidance on calibration workflows using Au nanoparticles as reference standards.


# Calibration Best Practices and Troubleshooting

## Overview
This notebook provides practical guidance on calibration workflows, common pitfalls, and how to validate your calibration results. We use Au nanoparticles as our reference material—a standard choice in electron microscopy for detector calibration due to their well-defined FCC crystal structure and high scattering power.

### What You'll Learn
1. Systematic approach to calibration using reference materials
2. How to handle imperfect or noisy data
3. Common calibration mistakes and fixes
4. Quality assessment metrics and validation
5. Best practices for reproducible calibration
6. Using Au nanoparticles as a calibration standard

## Part 1: The Calibration Workflow

In [1]:
import py4DSTEM
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime

print("\n" + "="*60)
print("CALIBRATION WORKFLOW CHECKLIST")
print("="*60)

workflow = {
    '1. Data Collection': [
        '☐ Use well-crystallized reference material (Si, Au, MoS₂, etc.)',
        '☐ Collect data with good signal-to-noise ratio',
        '☐ Ensure stable microscope conditions during data collection',
        '☐ Record microscope parameters (voltage, magnification, etc.)',
    ],
    '2. Pre-processing': [
        '☐ Load data and inspect for artifacts',
        '☐ Check for detector saturation or dead pixels',
        '☐ Perform noise reduction if needed',
        '☐ Verify data dimensions and data type',
    ],
    '3. Feature Identification': [
        '☐ Identify Bragg spots or other features',
        '☐ Measure peak positions with subpixel accuracy',
        '☐ Estimate confidence in feature detection',
        '☐ List known lattice parameters for reference',
    ],
    '4. Calibration Calculation': [
        '☐ Use detected features with known crystal data',
        '☐ Solve for origin, pixel size, and rotation',
        '☐ Check residuals and fitting quality',
        '☐ Validate with multiple features',
    ],
    '5. Validation': [
        '☐ Check symmetry of detected features',
        '☐ Compare with literature values',
        '☐ Test on multiple scan positions',
        '☐ Estimate uncertainty in calibration',
    ],
    '6. Documentation & Storage': [
        '☐ Save calibration parameters to file',
        '☐ Document method and reference material',
        '☐ Record date and microscope conditions',
        '☐ Include uncertainty estimates',
    ],
}

for section, items in workflow.items():
    print(f"\n{section}")
    for item in items:
        print(f"  {item}")


CALIBRATION WORKFLOW CHECKLIST

1. Data Collection
  ☐ Use well-crystallized reference material (Si, Au, MoS₂, etc.)
  ☐ Collect data with good signal-to-noise ratio
  ☐ Ensure stable microscope conditions during data collection
  ☐ Record microscope parameters (voltage, magnification, etc.)

2. Pre-processing
  ☐ Load data and inspect for artifacts
  ☐ Check for detector saturation or dead pixels
  ☐ Perform noise reduction if needed
  ☐ Verify data dimensions and data type

3. Feature Identification
  ☐ Identify Bragg spots or other features
  ☐ Measure peak positions with subpixel accuracy
  ☐ Estimate confidence in feature detection
  ☐ List known lattice parameters for reference

4. Calibration Calculation
  ☐ Use detected features with known crystal data
  ☐ Solve for origin, pixel size, and rotation
  ☐ Check residuals and fitting quality
  ☐ Validate with multiple features

5. Validation
  ☐ Check symmetry of detected features
  ☐ Compare with literature values
  ☐ Test on mul

## Part 2: Load and Inspect Data Quality

In [2]:
# Load reference dataset
filepath = 'raw_data/SI_Au_calib.dm4'

try:
    dataset = py4DSTEM.io.import_file(filepath)
    print(f"✓ Dataset loaded successfully")
except Exception as e:
    print(f"✗ Error loading dataset: {e}")
    dataset = None

# Quality assessment
if dataset is not None:
    print(f"\n" + "="*50)
    print("DATA QUALITY ASSESSMENT")
    print("="*50)
    
    data = dataset.data.astype(float)
    
    print(f"\n1. Data Statistics:")
    print(f"   Shape: {data.shape}")
    print(f"   Data type: {data.dtype}")
    print(f"   Min value: {data.min():.0f}")
    print(f"   Max value: {data.max():.0f}")
    print(f"   Mean: {data.mean():.1f}")
    print(f"   Std dev: {data.std():.1f}")
    
    # Check for saturation
    max_val = np.iinfo(np.uint16).max if dataset.data.dtype == np.uint16 else np.iinfo(np.uint8).max
    saturated_pixels = np.sum(dataset.data == max_val)
    saturation_pct = 100 * saturated_pixels / data.size
    
    print(f"\n2. Detector Saturation Check:")
    print(f"   Saturated pixels: {saturated_pixels} ({saturation_pct:.3f}%)")
    
    if saturation_pct > 1:
        print(f"   ⚠ Warning: High saturation! Consider re-collecting with lower beam intensity.")
    else:
        print(f"   ✓ Saturation level acceptable")
    
    # Check for dead pixels
    zero_pixels = np.sum(data == 0)
    zero_pct = 100 * zero_pixels / data.size
    
    print(f"\n3. Dead Pixel Check:")
    print(f"   Zero-value pixels: {zero_pixels} ({zero_pct:.3f}%)")
    
    if zero_pct > 5:
        print(f"   ⚠ Warning: Many zero pixels detected. May need interpolation.")
    
    # Dynamic range
    signal_range = data.max() - data.min()
    snr_estimate = data.mean() / data.std() if data.std() > 0 else 0
    
    print(f"\n4. Signal Quality:")
    print(f"   Signal range: {signal_range:.0f}")
    print(f"   SNR estimate: {snr_estimate:.1f}")
    
    if snr_estimate > 10:
        print(f"   ✓ Good signal-to-noise ratio")
    elif snr_estimate > 5:
        print(f"   ⚠ Moderate SNR - may need careful feature detection")
    else:
        print(f"   ✗ Poor SNR - feature detection may be difficult")

✗ Error loading dataset: The given filepath: 'raw_data/SI_Au_calib.dm4' 
does not exist


## Part 3: Common Calibration Mistakes

In [3]:
print("\n" + "="*70)
print("COMMON CALIBRATION MISTAKES & HOW TO AVOID THEM")
print("="*70)

mistakes = {
    '1. Wrong Reference Material': {
        'problem': 'Using lattice parameters from wrong phase or orientation',
        'symptoms': 'Calibration values are off by factors like 2x or √2',
        'fix': [
            '- Confirm material composition (EDS, EELS)',
            '- Determine zone axis orientation (by pattern matching)',
            '- Verify lattice parameters from literature',
        ],
    },
    '2. Wrong Axes Assignment': {
        'problem': 'Confusing navigation vs signal dimensions',
        'symptoms': 'Virtual images are highly distorted or transposed',
        'fix': [
            '- Always check dataset.shape explicitly',
            '- Index as data[Qx, Qy, Rx, Ry] consistently',
            '- Test with known patterns first',
        ],
    },
    '3. Detector Dead Zone': {
        'problem': 'Feature detection fails in saturated bright field region',
        'symptoms': 'Cannot find central spot or features near origin',
        'fix': [
            '- Mask out the bright field region before peak finding',
            '- Use higher-order reflections instead',
            '- Look for symmetric features away from center',
        ],
    },
    '4. Subpixel Accuracy Issues': {
        'problem': 'Peak positions rounded to integer pixels',
        'symptoms': 'Calibration uncertainty larger than expected',
        'fix': [
            '- Use center-of-mass calculation instead of max pixel',
            '- Apply Gaussian fitting to peak regions',
            '- Use multiple peaks to average out errors',
        ],
    },
    '5. Assuming Linear Scaling': {
        'problem': 'Detector distortion or nonlinearity ignored',
        'symptoms': 'Calibration works for some features but not others',
        'fix': [
            '- Check if distortion is present (ellipticity)',
            '- Use polynomial fitting if needed',
            '- Apply distortion correction (advanced)',
        ],
    },
    '6. Ignoring Rotation/Tilt': {
        'problem': 'Not accounting for detector rotation or sample tilt',
        'symptoms': 'Features appear at wrong angles',
        'fix': [
            '- Measure rotation from symmetry of features',
            '- Apply rotation matrix to coordinates',
            '- Use multiple zone axes for 3D refinement',
        ],
    },
}

for mistake, details in mistakes.items():
    print(f"\n{mistake}")
    print(f"  Problem: {details['problem']}")
    print(f"  Symptoms: {details['symptoms']}")
    print(f"  Fix:")
    for fix in details['fix']:
        print(f"    {fix}")


COMMON CALIBRATION MISTAKES & HOW TO AVOID THEM

1. Wrong Reference Material
  Problem: Using lattice parameters from wrong phase or orientation
  Symptoms: Calibration values are off by factors like 2x or √2
  Fix:
    - Confirm material composition (EDS, EELS)
    - Determine zone axis orientation (by pattern matching)
    - Verify lattice parameters from literature

2. Wrong Axes Assignment
  Problem: Confusing navigation vs signal dimensions
  Symptoms: Virtual images are highly distorted or transposed
  Fix:
    - Always check dataset.shape explicitly
    - Index as data[Qx, Qy, Rx, Ry] consistently
    - Test with known patterns first

3. Detector Dead Zone
  Problem: Feature detection fails in saturated bright field region
  Symptoms: Cannot find central spot or features near origin
  Fix:
    - Mask out the bright field region before peak finding
    - Use higher-order reflections instead
    - Look for symmetric features away from center

4. Subpixel Accuracy Issues
  Problem

## Part 4: Test Calibration Quality

In [4]:
# Example calibration to test
test_calibrations = {
    'ideal': {'pixelsize': 0.0201, 'origin_x': 235.5, 'origin_y': 235.5},
    'poor': {'pixelsize': 0.030, 'origin_x': 230.0, 'origin_y': 240.0},
    'unknown': {'pixelsize': 0.0199, 'origin_x': 235.3, 'origin_y': 235.4},
}

print("\n" + "="*60)
print("CALIBRATION QUALITY ASSESSMENT")
print("="*60)

# Reference values (from known Si-Au data)
ref_pixelsize = 0.0201  # 1/nm per pixel
ref_origin = (235.5, 235.5)  # pixels (approximate center)

print(f"\nReference values (Si-Au standard):")
print(f"  Pixel size: {ref_pixelsize} 1/nm/px")
print(f"  Origin: {ref_origin}")

quality_metrics = {}

for name, calib in test_calibrations.items():
    # Calculate errors
    pixelsize_error = abs(calib['pixelsize'] - ref_pixelsize) / ref_pixelsize * 100
    origin_error = np.sqrt((calib['origin_x'] - ref_origin[0])**2 + 
                          (calib['origin_y'] - ref_origin[1])**2)
    
    quality_metrics[name] = {
        'pixelsize_error_pct': pixelsize_error,
        'origin_error_pix': origin_error,
    }
    
    print(f"\n{name.upper()}:")
    print(f"  Pixel size: {calib['pixelsize']:.4f} (error: {pixelsize_error:.2f}%)")
    print(f"  Origin: ({calib['origin_x']:.1f}, {calib['origin_y']:.1f}) (error: {origin_error:.2f} px)")
    
    # Quality assessment
    if pixelsize_error < 1 and origin_error < 1:
        print(f"  ✓ EXCELLENT - suitable for quantitative analysis")
    elif pixelsize_error < 2 and origin_error < 2:
        print(f"  ✓ GOOD - suitable for most applications")
    elif pixelsize_error < 5 and origin_error < 5:
        print(f"  ⚠ ACCEPTABLE - use with caution for quantitative work")
    else:
        print(f"  ✗ POOR - needs improvement")


CALIBRATION QUALITY ASSESSMENT

Reference values (Si-Au standard):
  Pixel size: 0.0201 1/nm/px
  Origin: (235.5, 235.5)

IDEAL:
  Pixel size: 0.0201 (error: 0.00%)
  Origin: (235.5, 235.5) (error: 0.00 px)
  ✓ EXCELLENT - suitable for quantitative analysis

POOR:
  Pixel size: 0.0300 (error: 49.25%)
  Origin: (230.0, 240.0) (error: 7.11 px)
  ✗ POOR - needs improvement

UNKNOWN:
  Pixel size: 0.0199 (error: 1.00%)
  Origin: (235.3, 235.4) (error: 0.22 px)
  ✓ EXCELLENT - suitable for quantitative analysis


## Part 5: Robust Calibration Storage

In [5]:
# Create a comprehensive calibration file with metadata

calibration_record = {
    'metadata': {
        'date_calibrated': datetime.now().isoformat(),
        'calibration_method': 'Bragg spot detection with Si-Au reference',
        'reference_material': 'Si-Au thin film heterostructure',
        'zone_axis': '[110]',
        'microscopist': 'Student Name',
        'microscope': 'FEI TitanX 200kV',
        'camera': 'Gatan K2 IS',
    },
    'experimental_conditions': {
        'accelerating_voltage': 200,  # kV
        'magnification': 1500000,
        'camera_length': 165,  # mm
        'beam_current': 0.5,  # pA
        'dwell_time': 0.1,  # ms
    },
    'calibration_parameters': {
        'origin_x': 235.5,  # pixels
        'origin_y': 235.5,  # pixels
        'reciprocal_pixelsize': 0.0201,  # 1/nm per pixel
        'real_pixelsize': 0.1,  # nm per pixel (for scan area)
        'rotation_angle': 0.0,  # degrees
        'ellipticity': 1.0,  # ratio
    },
    'quality_assessment': {
        'n_peaks_detected': 28,
        'n_peaks_used': 12,
        'pixelsize_error': 0.5,  # percent
        'origin_error': 0.2,  # pixels
        'quality_rating': 'EXCELLENT',
    },
    'uncertainties': {
        'pixelsize_uncertainty': 0.0001,  # 1/nm per pixel
        'origin_x_uncertainty': 0.3,  # pixels
        'origin_y_uncertainty': 0.3,  # pixels
    },
    'notes': [
        'Calibrated using 220 Si Bragg reflection',
        'Validated with 440 and 200 reflections',
        'Stable throughout entire data collection',
        'Suitable for strain mapping applications',
    ],
}

# Save to JSON
filename = 'calibration_record.json'

with open(filename, 'w') as f:
    json.dump(calibration_record, f, indent=2)

print(f"✓ Calibration record saved to {filename}")
print(f"\nCalibration Record:")
print(json.dumps(calibration_record, indent=2)[:500] + "...")

✓ Calibration record saved to calibration_record.json

Calibration Record:
{
  "metadata": {
    "date_calibrated": "2026-01-20T14:54:14.039584",
    "calibration_method": "Bragg spot detection with Si-Au reference",
    "reference_material": "Si-Au thin film heterostructure",
    "zone_axis": "[110]",
    "microscopist": "Student Name",
    "microscope": "FEI TitanX 200kV",
    "camera": "Gatan K2 IS"
  },
  "experimental_conditions": {
    "accelerating_voltage": 200,
    "magnification": 1500000,
    "camera_length": 165,
    "beam_current": 0.5,
    "dwell_time": 0...


## Part 6: Load and Apply Saved Calibration

In [ ]:
# Example: Loading saved calibration for a new dataset

with open('calibration_record.json', 'r') as f:
    saved_calib = json.load(f)

print("Loaded Calibration:")
print(f"  Method: {saved_calib['metadata']['calibration_method']}")
print(f"  Date: {saved_calib['metadata']['date_calibrated']}")
print(f"  Quality: {saved_calib['quality_assessment']['quality_rating']}")
print(f"\nParameters:")
for key, val in saved_calib['calibration_parameters'].items():
    print(f"  {key}: {val}")

print(f"\nUncertainties:")
for key, val in saved_calib['uncertainties'].items():
    print(f"  {key}: {val}")

## Summary: Best Practices Checklist

In [ ]:
print("\n" + "="*70)
print("CALIBRATION BEST PRACTICES CHECKLIST")
print("="*70)

best_practices = {
    'Before Calibration': [
        '✓ Use well-characterized reference material',
        '✓ Record detailed experimental conditions',
        '✓ Ensure good data quality (low noise, no saturation)',
        '✓ Collect multiple regions for validation',
    ],
    'During Calibration': [
        '✓ Use subpixel accuracy for peak detection',
        '✓ Identify multiple features for validation',
        '✓ Check for systematic errors or outliers',
        '✓ Document all assumptions and choices',
    ],
    'After Calibration': [
        '✓ Validate with independent methods',
        '✓ Quantify uncertainties',
        '✓ Save calibration with complete metadata',
        '✓ Test on actual experimental samples',
    ],
    'Reproducibility': [
        '✓ Document microscope settings used',
        '✓ Include calibration date and operator',
        '✓ Version control for calibration files',
        '✓ Periodic recalibration (e.g., after maintenance)',
    ],
}

for category, items in best_practices.items():
    print(f"\n{category}:")
    for item in items:
        print(f"  {item}")

print(f"\n" + "="*70)
print("Remember: Good calibration = Reliable quantitative analysis!")
print("="*70)